In [1]:
import json
import time
import random
import pymongo
import copy
import MyCommon
from pymongo import MongoClient
from collections import deque
import cassiopeia
from cassiopeia import riotapi
import numpy as np
from sklearn import preprocessing, cluster, decomposition
from scipy.cluster.vq import kmeans,vq
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans, AffinityPropagation
from sklearn.decomposition import FastICA, PCA
from sklearn import metrics
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

riotapi.set_region("KR")
riotapi.set_api_key("79428a9e-5d98-469b-9b9b-429c1a750d24")
riotapi.set_rate_limits((10, 10), (500, 600))
riotapi.print_calls(False)

In [2]:
champions = riotapi.get_champions()
dict_id2idx = dict()
dict_idx2id = dict()
dict_id2key = dict()
dict_key2idx = dict()
dict_idx2key = dict()
for a_idx, a_champion in enumerate(champions):
    champ_id = a_champion.id
    champ_key = a_champion.key
    dict_id2idx[champ_id] = a_idx
    dict_idx2id[a_idx] = champ_id
    dict_id2key[champ_id] = champ_key
    dict_key2idx[champ_key.lower()] = a_idx
    dict_idx2key[a_idx] = champ_key.lower()
    
#print dict_id2key[dict_idx2id[132]]
#print dict_id2key

def GetIdxFromId(a_id):
    return dict_id2idx[a_id]

def GetIdFromIdx(a_idx):
    return dict_idx2id[a_idx]

def GetNameFromId(a_id):
    return dict_id2key[a_id]

def TranslateName(a_key):
    if a_key == "wukong":
        return "monkeyking"
    return a_key

print dict_key2idx

{u'evelynn': 125, u'zed': 127, u'nunu': 95, u'swain': 63, u'taric': 130, u'skarner': 91, u'ekko': 100, u'nasus': 67, u'graves': 27, u'nocturne': 77, u'shaco': 2, u'morgana': 72, u'aatrox': 104, u'draven': 105, u'nidalee': 4, u'kassadin': 37, u'sona': 38, u'monkeyking': 0, u'reksai': 116, u'cassiopeia': 42, u'taliyah': 81, u'malzahar': 112, u'amumu': 84, u'khazix': 47, u'olaf': 48, u'sivir': 17, u'riven': 18, u'nami': 119, u'jhin': 120, u'kogmaw': 102, u'gangplank': 20, u'syndra': 50, u'poppy': 22, u'volibear': 55, u'kennen': 57, u'ryze': 59, u'gragas': 126, u'garen': 94, u'shen': 60, u'karthus': 23, u'jayce': 24, u'bard': 64, u'sion': 65, u'elise': 97, u'alistar': 98, u'leblanc': 132, u'tahmkench': 106, u'ezreal': 133, u'fiddlesticks': 68, u'lux': 30, u'fiora': 33, u'kalista': 35, u'twistedfate': 69, u'kled': 6, u'illaoi': 9, u'kindred': 41, u'azir': 79, u'drmundo': 51, u'tryndamere': 13, u'hecarim': 46, u'lissandra': 113, u'darius': 12, u'yorick': 15, u'jax': 1, u'jarvaniv': 118, u'as

In [5]:
#Get pick rate for champions in each position
CHAMPIONS_STATS_URL = "https://oce.op.gg/champion/statistics"
r = requests.get(CHAMPIONS_STATS_URL)
soup = BeautifulSoup(r.text)
champions_part = soup.find("div", class_="tabWrap_recognized")


/usr/lib/python2.7/dist-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [3]:
CHAMPION_MASTER_URL = "https://www.op.gg/ranking/champions/name=%s"
for a_key in dict_key2idx:
    champion_url = CHAMPION_MASTER_URL % a_key
    print champion_url
    r = requests.get(champion_url)
    soup = BeautifulSoup(r.text)
    break

https://www.op.gg/ranking/champions/name=evelynn
